In [1]:
import os, sys
sys.path.append("../../../")

from src.core.module import Module
from src.core.losses import CrossEntropy
from src.core.optim import AdamW
from src.core.tensor import Tensor
import numpy as np


class Net(Module):
    def __init__(self, d_model, n_heads, vocab_size, max_seq_len, pad_idx=0):
        super().__init__()
        self.e = self.embedding(vocab_size, d_model, max_seq_len, pad_idx, name="Embedding")

        self.head1 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head2 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head3 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head4 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head5 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head6 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head7 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.head8 = self.transformer(d_model=d_model, n_heads=n_heads)
        self.project = self.linear(d_model, vocab_size, name="project")
    
    def forward(self, idx):
        x, padding_mask = self.e.get_sentence_embedding(idx)
        x = self.head1(x, padding_mask)
        x = self.head2(x, padding_mask)
        x = self.head3(x, padding_mask)
        x = self.head4(x, padding_mask)
        x = self.head5(x, padding_mask)
        x = self.head6(x, padding_mask)
        x = self.head7(x, padding_mask)
        x = self.head8(x, padding_mask)
        x = self.project(x)
        return x

    def train(self, x, y, epochs, optimizer):
        for epoch in range(epochs):
            y_hat = self.forward(x)
            loss = CrossEntropy(y_hat, y, axis=-1)
    
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if epoch % 1 == 0:
                print(f"Epoch {epoch}, Loss: {loss.data}")
                

        

In [2]:
if __name__ == "__main__":
    D_MODEL = 512
    VOCAB_SIZE = 100
    N_HEADS = 8
    MAX_SEQ_LEN = 1024
    PAD_IDX = 0

    model = Net(d_model=D_MODEL, n_heads=N_HEADS, vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, pad_idx=PAD_IDX)
    optimizer = AdamW(model.parameters(), lr=0.01)

    x = np.array([[1, 2, 3, 4, 5]])
    y = Tensor(np.array([[2, 3, 4, 5, 6]]), requires_grad=True)
    model.train(x, y, epochs=10, optimizer=optimizer)

    model.save_checkpoint(optimizer, "test_transformer")

    

Epoch 0, Loss: 4.609375
Epoch 1, Loss: 1.22265625
Epoch 2, Loss: 0.04901123046875
Epoch 3, Loss: 0.0023403167724609375
Epoch 4, Loss: -0.0
Epoch 5, Loss: -0.0
Epoch 6, Loss: -0.0
Epoch 7, Loss: -0.0
Epoch 8, Loss: -0.0
Epoch 9, Loss: -0.0


In [3]:
if __name__ == "__main__":
    new_model = Net(d_model=D_MODEL, n_heads=N_HEADS, vocab_size=VOCAB_SIZE, max_seq_len=MAX_SEQ_LEN, pad_idx=PAD_IDX)
    new_optimizer = AdamW(new_model.parameters(), lr=0.01)

    new_model.load_checkpoint(new_optimizer, "test_transformer")

    model.train(x, y, epochs=10, optimizer=new_optimizer)


FileNotFoundError: [Errno 2] No such file or directory: 'test_transformer/model/transformer_8_layernorm_2_gamma.npy'